In [1]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import numpy as np
import asyncio
import threading
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Shared variables to store the latest frame and flag indicating frame change
latest_frame = None
frame_changed = False

def generate_frames():
    global latest_frame
    global frame_changed
    num_frames=0

    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()

        if ret:
            
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame
            frame_changed = True

            

def start_frame_generation():
    # Start the frame generation in a separate thread
    threading.Thread(target=generate_frames, daemon=True).start()

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global frame_changed

        while True:
            if frame_changed==True:  # Check if there's a latest frame available
                yield (
                    b'--frame\r\n'
                    b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                )
                frame_changed = False
            else:
                await asyncio.sleep(0.02)  # If no latest frame available, wait for a short time

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    start_frame_generation()  # Start frame generation in a separate thread
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [13196]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50829 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:50830 - "GET /video_feed HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13196]


In [6]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import asyncio
import threading
import nest_asyncio
from queue import Queue

nest_asyncio.apply()
app = FastAPI()

# Asynchronous queue for frame storage
frame_queue = Queue(maxsize=1)  # Only store the latest frame

# Global lock for thread safety while accessing the queue
frame_lock = threading.Lock()

def generate_frames():
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # Read and process the frame
        ret, frame = cap.read()
        if ret:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            latest_frame = buffer.tobytes()

            # Store the latest frame in the queue, replacing the previous one
            with frame_lock:
                if frame_queue.full():
                    frame_queue.get_nowait()  # Remove the old frame
                frame_queue.put(latest_frame)

        time.sleep(0.02)  # Adjust frame capture rate as needed

def start_frame_generation():
    # Start the frame generation in a separate thread
    threading.Thread(target=generate_frames, daemon=True).start()

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        while True:
            with frame_lock:
                if not frame_queue.empty():
                    frame = frame_queue.get_nowait()
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n'
                    )
            await asyncio.sleep(0.02)  # Adjust frame sending rate as needed

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    start_frame_generation()  # Start frame generation in a separate thread
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [1700]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55996 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:55999 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56000 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56004 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56005 - "GET /video_feed HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1700]


In [1]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import asyncio
import threading
import nest_asyncio
from queue import Queue

nest_asyncio.apply()
app = FastAPI()

# Asynchronous queue for frame storage
frame_queue = Queue(maxsize=1)  # Only store the latest frame

# Global lock for thread safety while accessing the queue
frame_lock = threading.Lock()

async def generate_frames():
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # Read and process the frame
        ret, frame = cap.read()
        if ret:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            latest_frame = buffer.tobytes()

            # Store the latest frame in the queue, replacing the previous one
            with frame_lock:
                if frame_queue.full():
                    frame_queue.get_nowait()  # Remove the old frame
                frame_queue.put(latest_frame)

        await asyncio.sleep(0.02)  # Adjust frame capture rate as needed

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        while True:
            with frame_lock:
                if not frame_queue.empty():
                    frame = frame_queue.get_nowait()
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n'
                    )
            await asyncio.sleep(0.02)  # Adjust frame sending rate as needed

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    asyncio.create_task(generate_frames())  # Start frame generation as a coroutine
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [13188]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56112 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56131 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56132 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56151 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56150 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56204 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56205 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56216 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56219 - "GET /video_feed HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13188]


In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

frame_queue = asyncio.Queue()
frame_event = asyncio.Event()

async def generate_frames(start_time):
    global frame_queue
    global frame_event
    c = 0
    frame_interval = 1 / 4
    num_frames = 0
    url = "http://localhost:3000/video_feed" # Update the URL if needed
    cap = cv2.VideoCapture(url)
    
    # Seek to the start time
    while time.time() - start_time < 0:
        await asyncio.sleep(0.1)
    
    while True:
        frame_start_time = time.time()
        ret, frame = cap.read()
        
        if ret:
            _, buffer = cv2.imencode('.jpg', frame)
            await frame_queue.put(buffer.tobytes())
            frame_event.set()
        
        num_frames += 1
        frame_time = time.time() - frame_start_time
        delay = frame_interval - frame_time

        try:
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global frame_queue
        global frame_event
        start_time = time.time()
        
        # Start generating frames from the current time
        asyncio.create_task(generate_frames(start_time))
        
        while True:
            await frame_event.wait()
            frame_event.clear()
            frame = await frame_queue.get()
            yield (
                b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n'
            )
    
    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [12308]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56992 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:56991 - "GET /video_feed HTTP/1.1" 200 OK


In [1]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
nest_asyncio.apply()

app = FastAPI()

frame_queue = Queue(maxsize=1)
frame_event = asyncio.Event()
last_request_time = None

async def generate_frames():
    global frame_queue
    global frame_event
    global last_request_time
    c = 0
    frame_interval = 1 / 4
    start_time = time.time()
    num_frames = 0
    url = "http://localhost:3000/video_feed" # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        frame_start_time = time.time()
        ret, frame = cap.read()
        
        if ret:
            _, buffer = cv2.imencode('.jpg', frame)
            # Store only the latest frame in the queue
            if frame_queue.full():
                frame_queue.get_nowait()  # Discard the previous frame
            frame_queue.put(buffer.tobytes())
            frame_event.set()
        
        elapsed_time = time.time() - start_time
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time

        frame_time = time.time() - frame_start_time
        delay = frame_interval - frame_time

        try:
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break

@app.get("/video_feed")
async def video_feed():
    global last_request_time
    last_request_time = time.time()
    async def stream_frame():
        global frame_queue
        global frame_event
        while True:
            await frame_event.wait()
            frame_event.clear()
            frame = frame_queue.get()
            yield (
                b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n'
            )
    
    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [14104]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56488 - "GET /video_feed HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14104]


In [1]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

frame_queue = asyncio.Queue(maxsize=1)
frame_event = asyncio.Event()

async def generate_frames(start_time):
    global frame_queue
    global frame_event
    c = 0
    frame_interval = 1 / 4
    num_frames = 0
    url = "http://localhost:3000/video_feed" # Update the URL if needed
    cap = cv2.VideoCapture(url)
    
    # Seek to the start time
    while time.time() - start_time < 0:
        await asyncio.sleep(0.1)
    
    while True:
        frame_start_time = time.time()
        ret, frame = cap.read()
        
        if ret:
            _, buffer = cv2.imencode('.jpg', frame)
            await frame_queue.put(buffer.tobytes())
            frame_event.set()
        
        num_frames += 1
        frame_time = time.time() - frame_start_time
        delay = frame_interval - frame_time

        try:
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global frame_queue
        global frame_event
        start_time = time.time()
        
        # Start generating frames from the current time
        asyncio.create_task(generate_frames(start_time))
        
        while True:
            await frame_event.wait()
            frame_event.clear()
            frame = await frame_queue.get()
            yield (
                b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n'
            )
    
    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [1684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1684]
